In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
from bs4 import BeautifulSoup

In [3]:
# Webscraper
url = 'https://www.bing.com/images/search?sp=-1&ghc=1&pq=male+face&sc=8-9&sk=&cvid=DB6082808D684A4A9104F772BD599D39&q=male+face&qft=+filterui:license-L2_L3_L5_L6&FORM=IRFLTR'

# Query
headers = {'user-agent': 'scraper - school project (bengriffith@outlook.com)'}
response = requests.get(url, headers=headers)

if response.ok:
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    raw_images = soup.select('.content')

In [23]:
img_tag = soup.findAll('img')
    
images = [img['src']  for img in img_tag]

In [27]:
i = 0
for image in images[1:15]:
    with open('sample' + str(i) + '.png', 'wb') as f:
        response = requests.get(image)
        f.write(response.content)
    i += 1